#
# Create a multi-signature wallet using Bitcoinlib testnet and then create a transaction
#


## import libraries and testnest

bitcoinlib.test.sqlite is belong to bitcoinlib library


In [1]:
import os
from pprint import pprint
from bitcoinlib.wallets import *

test_databasefile = os.path.join(BCL_DATABASE_DIR, 'bitcoinlib.test.sqlite')
test_database = 'sqlite:///' + test_databasefile
if os.path.isfile(test_databasefile):
    os.remove(test_databasefile)

print(test_database)


sqlite:///C:\Users\Klaus\.bitcoinlib\database\bitcoinlib.test.sqlite


## Create 3 wallets with one private keys each, and 2 public keys corresponding with other wallets

In [2]:
NETWORK = 'bitcoinlib_test' # Bitcoin test network.

#Create 3 new private key. HDKey is a "hierarchical deterministic key" for use in BIP32 wallets.
pk1 = HDKey(network=NETWORK) 
pk2 = HDKey(network=NETWORK)
pk3 = HDKey(network=NETWORK)

#Create 3 new wallets, each with a different private key.
## w1 with private key pk1 and public key corresponding to the second and third private key
klist = [pk1, pk2.public_master_multisig(), pk3.public_master_multisig()]
wl1 = Wallet.create('multisig_2of3_cosigner1', sigs_required=2, keys=klist,
                    network=NETWORK, db_uri=test_database)

## w2 with private key pk2 and public key corresponding to the first and third private key
klist = [pk1.public_master_multisig(), pk2, pk3.public_master_multisig()]
wl2 = Wallet.create('multisig_2of3_cosigner2',  sigs_required=2, keys=klist,
                    network=NETWORK, db_uri=test_database)

## w3 with private key pk3 and public key corresponding to the first and second private key
klist = [pk1.public_master_multisig(), pk2.public_master_multisig(), pk3]
wl3 = Wallet.create('multisig_2of3_cosigner3', sigs_required=2, keys=klist,
                    network=NETWORK, db_uri=test_database)

## Generate a new key in each wallet, all these keys should be the same

In [3]:
nk1 = wl1.new_key(cosigner_id=1)
nk2 = wl2.new_key(cosigner_id=1)
nk3 = wl3.new_key(cosigner_id=1)
assert nk1.wif == nk2.wif == nk3.wif # make sure they are the same
print("Created new multisig address: ", nk1.wif)


Created new multisig address:  multisig-<Address(address=235xxsdL4WHAm7X6UcsuzCQKps7yLCGGrzV)>


## Create a transaction

In [4]:
fee = 29348 # fee in satoshi

#updates the Unspent Transaction Outputs (UTXOs) for wl1, which is a wallet object
wl1.utxos_update()  # On bitcoinlib testnet, this automatically creates an UTXO
utxo = wl1.utxos()[0] # get the first UTXO available
print("UTXO: ", utxo)

# Defining Output and Input for the Transaction
# + Output: The address to send the transaction to and the amount to send
# + Input: The UTXO to use as input for the transaction: the transaction id, the output number, the key id and the value
output_arr = [('23Gd1mfrqgaYiPGkMm5n5UDRkCxruDAA8wo', utxo['value'] - fee)]
input_arr = [(utxo['txid'], utxo['output_n'], utxo['key_id'], utxo['value'])]

# Create a transaction for wallet: wl1
t = wl1.transaction_create(output_arr, input_arr, fee=fee)


UTXO:  {'key_id': 46, 'output_n': 0, 'script': b'', 'value': 100000000, 'spending_txid': None, 'transaction_id': 1, 'address': '235xxsdL4WHAm7X6UcsuzCQKps7yLCGGrzV', 'script_type': 'p2sh', 'spent': False, 'spending_index_n': None, 'confirmations': 10, 'txid': '2e1d6f86cae8d1d1cc2284883dad563bd2707728ccbb3260c53ba9dc69b1a3e4', 'network_name': 'bitcoinlib_test'}


## sign transaction with first wallet, it should not verify yet

In [5]:
# Sign the transaction with the private key of the first cosigner
t.sign()

#display the transaction information
pprint(t.as_dict())

#verify the transaction (with unverified Signature)
print("Verified (should be False): ", t.verify())

{'block_hash': None,
 'block_height': None,
 'coinbase': False,
 'confirmations': None,
 'date': None,
 'fee': 29348,
 'fee_per_kb': 79105,
 'flag': None,
 'input_total': 100000000,
 'inputs': [{'address': '235xxsdL4WHAm7X6UcsuzCQKps7yLCGGrzV',
             'compressed': True,
             'double_spend': False,
             'encoding': 'base58',
             'index_n': 0,
             'locktime_cltv': None,
             'locktime_csv': None,
             'output_n': 0,
             'prev_txid': '2e1d6f86cae8d1d1cc2284883dad563bd2707728ccbb3260c53ba9dc69b1a3e4',
             'public_hash': '536bbfb7fd804b3c702aeff9bfd9c8bafd393666',
             'public_keys': ['03423572ee8447033b3ba3ff446fc5dc25a3b229565b1ff1faba1e4a2e451ecb08',
                             '03b5fcf9e28cbe207ab214711746026663e1aaa22e4c15f0c10c8551837d73dc31',
                             '03d4d37728a9680c2badb20f4bac56fa965433af0977ef81547f3061f866ac28e0'],
             'redeemscript': '522103423572ee8447033b3ba3ff446

## Import transaction (with first signature) in 3rd wallet and sign with wallet's private key

In [6]:
#update the UTXOs for wl3
wl3.utxos_update()

#Import the transaction t into wl3: this will add the transaction to the database and update the UTXOs
t2 = wl3.transaction_import(t)

#verify the transaction (with verified Signature)
t2.sign()

#check if the transaction is valid
print("Verified (should be True by now): ", t2.verify())

Verified (should be True by now):  True


#
# Create Multisig 2-of-2 testnet wallet, and sweep all UTXO's
#

## Create 2 cosigner multisig wallets

In [7]:
NETWORK = 'bitcoinlib_test' # Bitcoin test network.

#Create 2 new private key.
pk1 = HDKey(network=NETWORK)
pk2 = HDKey(network=NETWORK)

#Create 2 new wallets, each with a different private key.
wl1 = Wallet.create('multisig_2of2_cosigner1', sigs_required=2,
                    keys=[pk1, pk2.public_master_multisig()],
                    network=NETWORK, db_uri=test_database)
wl2 = Wallet.create('multisig_2of2_cosigner2', sigs_required=2,
                    keys=[pk1.public_master_multisig(), pk2],
                    network=NETWORK, db_uri=test_database)

#Create a new key for each wallet
nk1 = wl1.new_key()
nk2 = wl2.new_key(cosigner_id=0) # cosigner_id=0 means that this is the first cosigner

## Create a transaction for send within the wallet 1

In [8]:

# wl1.utxos_update() # Update UTXO's in wallet 1
# utxos = wl1.utxos() # Get UTXO's in wallet 1
# if not utxos: # If no UTXO's found, print address to deposit testnet bitcoin
#     print("Deposit testnet bitcoin to this address to create transaction: ", nk1.address)
# else: # If UTXO's found, sweep wallet
#     print("Utxo's found, now sweep wallet")

#     # Sweep wallet 1
#     # sweep help create and broadcast a transaction that consolidates all UTXOs in the wallet (wl1) and sends the funds to a new address within the same wallet
#     res = wl1.sweep(wl1.new_key().address, min_confirms=0)
#     assert res.txid # expect a transaction ID, indicating that the sweep operation was successful, and a transaction ID (txid) was generated
#     print("Transaction ID: ", res.txid)


#     print("UTXOs in wl2 before update: ", wl2.utxos())  # Debug print
#     wl2.utxos_update()# Update UTXO's in wallet 2 :This is important because the sweep operation in wl1 might create new UTXOs, and wl2 needs to be aware of them
#     wl2.new_key() # Create a new key in wallet 2 for the change address
#     print("UTXOs in wl2 afterupdate: ", wl2.utxos())  # Debug print
#     print("Keys in wl2: ", wl2.keys())  # Debug print
    
#     #Imports the sweep transaction into wl2
#     t2 = wl2.transaction_import(res)

#     #Signs the transaction with the private key of the first cosigner
#     t2.sign()
#     print("Verified (should be True): ", t2.verify())

#     #Pushes the transaction to the network
#     t2.send()
#     print("Push transaction result: ", t2.status)

Utxo's found, now sweep wallet
Transaction ID:  7b3c54e2dc522cba7b9e17bc255f89b5b59e50ed0f46b99677015ba9862ca481
UTXOs in wl2 before update:  []
UTXOs in wl2 afterupdate:  [{'key_id': 86, 'output_n': 0, 'script': b'', 'value': 100000000, 'spending_txid': None, 'transaction_id': 7, 'address': '23BZPP6YcvAcCVuQp8uQjj7jCEUn4vSrK4N', 'script_type': 'p2sh', 'spent': False, 'spending_index_n': None, 'confirmations': 10, 'txid': '7ff6da3ec9872aa4f7f87f2ad2fe54589a70948d456443e0814a5232825f4ef5', 'network_name': 'bitcoinlib_test'}, {'key_id': 86, 'output_n': 0, 'script': b'', 'value': 100000000, 'spending_txid': None, 'transaction_id': 8, 'address': '23BZPP6YcvAcCVuQp8uQjj7jCEUn4vSrK4N', 'script_type': 'p2sh', 'spent': False, 'spending_index_n': None, 'confirmations': 10, 'txid': 'fad43b514ccbf03a412019239d29c304ab3b4164bf33513b5946b6f54eba2c0b', 'network_name': 'bitcoinlib_test'}]
Keys in wl2:  [<DbKey(id='86', name='Multisig Key 85/74', wif='multisig-<Address(address=23BZPP6YcvAcCVuQp8uQjj7j

WalletError: UTXO 7b3543534977d61f5ee0de5b646568304a236e77afa20d3037b4ca69fedfd1df and key with address 231uhTVVSRmcxk8MGJVkkpaYh3aUak3Se7U not found in this wallet

## create a transaction with 2 signatures and broadcast it to specific address

In [ ]:
wl1.utxos_update()

# Specify the destination address and amount in satoshis
fee = 29348 # fee in satoshi
utxo = wl1.utxos()[0] # get the first UTXO available
output_arr = [('23Gd1mfrqgaYiPGkMm5n5UDRkCxruDAA8wo', 500000)]
input_arr = [(utxo['txid'], utxo['output_n'], utxo['key_id'], utxo['value'])]

# Get UTXOs from the wallet (assuming they exist)
wl1.utxos_update()
utxos = wl1.utxos()

if not utxos:
    print("No UTXOs found, deposit testnet bitcoin to this address to create transaction: ", nk1.address)
else:
    print("UTXOs found, now create transaction")
    # Create the transaction
    t = wl1.transaction_create(output_arr, input_arr, fee=fee)
    # Sign the transaction
    t.sign()
    # Verify the transaction
    print("Verified (should be False beacause wallet not sign yet): ", t.verify())
    # Push the transaction to the network
    t.send()
    print("Push transaction result: ", t.status)

    # send to wallet 2 to sign
    print("UTXOs in wl2 before update: ", wl2.utxos())  # Debug print
    wl2.utxos_update()# Update UTXO's in wallet 2 :This is important because the sweep operation in wl1 might create new UTXOs, and wl2 needs to be aware of them
    wl2.new_key() # Create a new key in wallet 2 for the change address
    print("UTXOs in wl2 afterupdate: ", wl2.utxos())  # Debug print
    print("Keys in wl2: ", wl2.keys())  # Debug print
    
    #Imports the sweep transaction into wl2
    t2 = wl2.transaction_import(t)

    #Signs the transaction with the private key of the first cosigner
    t2.sign()
    print("Verified (should be True because both wallet are signed now): ", t2.verify())

    #Pushes the transaction to the network
    t2.send()
    print("Push transaction result: ", t2.status)

IndexError: list index out of range

#
# Multisig wallet using single keys for cosigner wallet instead of BIP32 type key structures
#

this means we only need one private key to sign a transaction, this will simpler than 2-of-2 multisig wallet above

In [ ]:
NETWORK = 'bitcoinlib_test'
pk1 = HDKey('YXscyqNJ5YK411nwB33KeVkhSVjwwUkSG9xG3hkaoQFEbTwNJSrNTfni3aSSYiKtPeUPrFLwDsqHwZjNXhYm2DLEkQoaoikHoK2emrHv'
            'mqSEZrKP', network=NETWORK)
pk2 = HDKey('YXscyqNJ5YK411nwB3kXiApMaJySYss8sMM9FYgXMtmQKmDTF9yiu7yBNKnVjE8WdVVvuhxLqS6kHvW2MPHKmYzbzEHQsDXXAZuu1rCs'
            'Hcp7rrJx', network=NETWORK, key_type='single')
wl1 = Wallet.create('multisig_single_keys1', [pk1, pk2.public()],
                    sigs_required=2, network=NETWORK, db_uri=test_database)
wl2 = Wallet.create('multisig_single_keys2', [pk1.public_master_multisig(), pk2],
                    sigs_required=2, network=NETWORK, db_uri=test_database)

## Create multisig keys and update UTXO's

In [ ]:
# create a new key for each wallet and indicate that anyone can be the cosigner
wl1.new_key(cosigner_id=0)
wl2.new_key(cosigner_id=0)

# update the UTXOs for each wallet
wl1.utxos_update()
wl2.utxos_update()

2

## Create transaction and sign with both wallets, return address should be the same

In [ ]:
# create a transaction "t" in waller 2
t = wl2.transaction_create([('23Gd1mfrqgaYiPGkMm5n5UDRkCxruDAA8wo', 5000000)])
t.sign() # sign the transaction with the private key(s) associated with wl2

# import the transaction into wl1
t2 = wl1.transaction_import(t)
t2.sign()# sign the transaction with the private key(s) associated with wl1 and update the UTXOs

# verify the transaction
print("%s == %s: %s" % (t.outputs[1].address, t2.outputs[1].address, t.outputs[1].address == t2.outputs[1].address))
print("Verified (should be True): ", t2.verify())

23Gd1mfrqgaYiPGkMm5n5UDRkCxruDAA8wo == 23Gd1mfrqgaYiPGkMm5n5UDRkCxruDAA8wo: True
Verified (should be True):  True
